### Script to run all prompts on the data set ###

In [1]:
import os
import pandas as pd
import numpy as np
import logging
import time
from pathlib import Path

logger = logging.getLogger(__name__)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import process_prompt
from llmt.openai import OpenAIModel, OpenAI
from llmt.openai import MentalHealth, OutpatientServices, InpatientServices, create_messages
from llmt.performance import Performance

In [2]:
# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
experiment_name = 'hcp_experiment_03'
output_dir = os.path.join(data_dir, experiment_name)
Path(output_dir).mkdir(exist_ok=True, parents=True)

test_file_name = 'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df_all = pd.read_parquet(test_file)
# Filter the labeled data
df_train = df_all.loc[df_all['dset'] == 'train'].\
                reset_index(drop=True).\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})
display(df_train.head())
print(df_train.shape)

,id,name,description,mental_health,inpatient,outpatient,dset
0,431643-07,Actriv,Provider of healthcare staffing services based...,2,0,0,train
1,310749-31,Alima,Operator of a non-governmental organization in...,0,0,0,train
2,162054-28,Apothecare,Provider of pharmacy services intended to prov...,0,0,0,train
3,597285-28,April Health (Clinics/Outpatient Services),Provider of mental health services intended to...,1,0,1,train
4,373978-90,Arise Child and Family Service,Operator of independent living centers caterin...,2,0,0,train


(187, 7)


In [3]:
# Add some additional samples 
test_samples = 0
random_state = 111
df_test = df_all.loc[df_all['mental_health'].isnull()].\
                sample(n=test_samples, replace=False, random_state=random_state).\
                reset_index(drop=True)
# Combine the training and test samples
df = pd.concat([df_train, df_test], axis=0, ignore_index=True).\
                sample(frac=1, random_state=random_state).\
                reset_index(drop=True)
for dset in ['train', 'test']:
    print(f'{dset}: {len(df.loc[df['dset']==dset, 'id'].unique())}')

display(df.head())

train: 187
test: 0


,id,name,description,mental_health,inpatient,outpatient,dset
0,248187-88,River Crest Hospital,Provider of psychiatric diagnostic medical ser...,1.0,1.0,1.0,train
1,127477-00,Curry Health District,Operator of a healthcare network in Southern O...,0.0,1.0,1.0,train
2,11809-18,BrightSpring Health Services (NAS: BTSG),BrightSpring Health Services Inc is a home and...,2.0,0.0,0.0,train
3,144253-45,Bon Secours Baltimore Hospital,Operator of a full-service hospital in Baltimo...,0.0,1.0,1.0,train
4,439172-02,East Tennessee Behavioral Health,Operator of an inpatient behavioral health hos...,1.0,1.0,2.0,train


In [12]:
temperature = 0 
runs = 1
results_file_base = f'250422_mh3ip1op1'

# Run the prompt on all data
company_id_list = sorted(list(df['id'].unique()))
start_time = time.perf_counter()

# Instantiate the model class
model = OpenAIModel()

for run in range(runs):
    execution_time = time.perf_counter() - start_time
    execution_time_min = np.round(execution_time/60, decimals=1)
    print(f'Execution time: {execution_time_min} minutes.')
    results_run_file_name = f'{results_file_base}_{str(run).zfill(2)}.parquet'
    results_run_file = os.path.join(output_dir, results_run_file_name)
    print(f'STARTING RUN {run + 1} / {runs}: {results_run_file_name}')
    results_run_df_list = []
    for c, company_id in enumerate(company_id_list):
        if (c + 1) % 20 == 0:
            print(f'Sending description {c + 1} / {len(company_id_list)} to the model')
    
        df_id = df.loc[df['id'] == company_id]
        name = df_id['name'].values[0]
        description = df_id['description'].values[0]
        
        # Mental health predictions
        response_mh = model.predict_mh(name=name, description=description, version=3, temperature=temperature)
        df_id = df_id.assign(**response_mh)
        
        # Inpatient predictions
        response_ip = model.predict_ip(name=name, description=description, version=1, temperature=temperature)
        df_id = df_id.assign(**response_ip)
        
        # Outpatient predictions
        response_op = model.predict_op(name=name, description=description, version=1, temperature=temperature)
        df_id = df_id.assign(**response_op)

        results_run_df_list.append(df_id)

    # Add the new data frame to the list
    results_run_df = pd.concat(results_run_df_list, axis=0, ignore_index=True)
    results_run_df = results_run_df.assign(temperature=temperature)
    # Save it
    results_run_df.to_parquet(results_run_file)

Execution time: 0.0 minutes.
STARTING RUN 1 / 1: 250422_mh3ip1op1_00.parquet
Sending description 20 / 187 to the model
Sending description 40 / 187 to the model
Sending description 60 / 187 to the model
Sending description 80 / 187 to the model
Sending description 100 / 187 to the model
Sending description 120 / 187 to the model
Sending description 140 / 187 to the model
Sending description 160 / 187 to the model
Sending description 180 / 187 to the model


In [18]:
# Load the data and check the results
# results_file_name = 'inpatient_01_results.parquet'
results_run_file_name = f'{results_file_base}_{str(run).zfill(2)}.parquet'
results_run_file = os.path.join(output_dir, results_run_file_name)
df = pd.read_parquet(results_run_file)

results_train = df.loc[df['dset'] == 'train']

# Mapping true variable and predictions
variable_dict = {'mental_health': 'pred_mh',
                 'inpatient': 'pred_ip',
                 'outpatient': 'pred_op'}

for true_col in variable_dict.keys():
    print(f'PERFORMANCE: {true_col.upper()}')
    stat_dict = Performance(data=df.copy()).\
            binary_performance(true_col=true_col, pred_col=variable_dict.get(true_col))
    stat_df = pd.DataFrame(stat_dict, index=[0])
    display(stat_df)
    print()

PERFORMANCE: MENTAL_HEALTH


,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,136,35,110,29,6,26,0.8088,0.9483,0.7953,0.8286,0.873



PERFORMANCE: INPATIENT


,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,69,113,68,103,10,1,0.9855,0.8718,0.3791,0.9115,0.9252



PERFORMANCE: OUTPATIENT


,p,n,tp,tn,fp,fn,recall,precision,min_precision,specificity,f_score
0,94,42,76,31,11,18,0.8085,0.8736,0.6912,0.7381,0.8398


### Look at the false negatives which should be positive ###

In [10]:
id_list = df.loc[(df['mental_health'] == 1) & (df['pred_mh'] == 0), 'id']
print(len(id_list))
for company_id in id_list:
    df_id = df.loc[df['id'] == company_id]
    name = df_id['name'].values[0]
    description = df_id['description'].values[0]
    display(df_id)
    print(name.upper())
    print(description)
    print()

41


,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,temperature
1,10127-62,Quest Diagnostics (NYS: DGX),Quest Diagnostics is a leading independent pro...,1.0,0.0,0.0,train,0,0,0,1,0


QUEST DIAGNOSTICS (NYS: DGX)
Quest Diagnostics is a leading independent provider of diagnostic testing, information, and services in the us. The company generates over 95% of its revenue through clinical testing, anatomic pathology, esoteric testing, and substance abuse testing with specimens collected at its national network of roughly 2,300 patient service centers, as well as multiple doctors offices and hospitals. The firm also runs a much smaller diagnostic solutions segment that provides clinical trials testing, risk-assessment services, and information technology solutions.



,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,temperature
2,10195-21,IASIS Healthcare,Owner and operator of medium-sized acute care ...,1.0,1.0,1.0,train,0,1,1,1,0


IASIS HEALTHCARE
Owner and operator of medium-sized acute care hospitals. The company's hospitals offers highly motivated and compassionate physicians, nurses and other professionals providing value-based services to patients, employers, physicians and payors through their integrated healthcare providers and health plans to improve the quality of life for the communities.



...

OUTRO HEALTH
Developer of digital health platform designed to discontinue psychiatric medications and provide a new understanding of mental health, through education and compassionate care. The company offers a comprehensive library, courses for every stage, holistic wellness practices, and access to knowledgeable providers, enabling individuals to safely and effectively manage their antidepressant withdrawal and promote holistic mental well-being.



,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,temperature
176,693780-04,CRH Healthcare (Urgent Care Facilities in Alab...,"Three Urgent Medcare located in Huntsville, Ma...",1.0,0.0,1.0,train,0,0,1,1,0


CRH HEALTHCARE (URGENT CARE FACILITIES IN ALABAMA)
Three Urgent Medcare located in Huntsville, Madison, and Meridianville, Alabama. These urgent cares offer services for illness, injury, diagnostic testing, COVID-19 tests, mental health, and other related healthcare needs, enabling patients to receive convenient and accessible treatment.



,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,temperature
180,718216-75,Self (Other Healthcare Technology Systems),Provider of mental health screening services i...,1.0,0.0,1.0,train,0,0,0,1,0


SELF (OTHER HEALTHCARE TECHNOLOGY SYSTEMS)
Provider of mental health screening services intended to improve the quality of mental health care. The company operates in an affiliate model, where they conduct screening for mental health conditions and offers recommendations for suitable service providers to patients, enabling mental health professionals to improve quality of care and reduce customer acquisition costs

